# Mini Lab: SVM and Logistic Regression Modeling

### Teammates: Chad Kwong, Erin McClure-Price, Alex Lopez, Chris Haub

## Initial loading of dataset and all related packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import linear_model
from sklearn import svm

#Suppress futurewarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Set Figure Size
plt.rcParams["figure.figsize"] = (20,10)



#Import Dataset from github repo
bankData = pd.read_csv('bank-additional-full.csv', sep=';', na_values="unknown")
#Optional: Add in index column
#bankData.insert(0, 'Sample_ID', range(1,len(bankData)+1))

#Creating variables for indexing continuous and categorical variables
conCol = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 
          'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
catCol = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
          'contact', 'month', 'day_of_week', 'poutcome', 'y']

In [2]:
bankData2 = bankData.dropna()
#remove 'default' column
bankData2.drop(['default'], axis=1)
bankData2.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
# checked for missing values, no missing values
bankData2.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [4]:
bankData2.shape

(30488, 21)

## Building Model

In [5]:
#model = linear_model.Regression()
#model.fit(bankData, )

In [6]:
#define the target variable
y = bankData2.y
y.head()

0    no
2    no
3    no
4    no
6    no
Name: y, dtype: object

In [7]:
# one hot encoding of ALL categorical variables

# pd.concat([*]], axis=1) // this line of code concatenates all the data frames in the [*] list
# [** for col in categ_features] // this steps through each feature in categ_features and 
#                                //   creates a new element in a list based on the output of **
# pd.get_dummies(df_imputed[col],prefix=col) // this creates a one hot encoded dataframe of the variable=col (like code above)

categ_features = ['job','marital','education','default','housing','loan','contact','month','poutcome'];

OneHotDF = pd.concat([pd.get_dummies(bankData2[col],prefix=col,drop_first=True) for col in categ_features], axis=1)

OneHotDF.head()

#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/01.%20Pandas.ipynb

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_nonexistent,poutcome_success
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
OneHotDF.columns

Index(['job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_married',
       'marital_single', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'default_yes', 'housing_yes', 'loan_yes', 'contact_telephone',
       'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar',
       'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [9]:
# pandas.pop
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pop.html
Y = bankData2.pop("y")

In [10]:
# merged OneHotDF and bankData files

X = pd.concat([bankData2.select_dtypes(exclude='object'),OneHotDF],axis=1)

In [11]:
#separating input data into two parts X (features) and Y (target)
# features = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 
#           'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'job', 'marital', 'education', 'default', 'housing', 'loan', 
#           'contact', 'month', 'day_of_week', 'poutcome']
# # Separating out the features
# X = mergedDF.loc[:, features].values

# #Separating out the target
# Y = mergedDF.loc[:, ['y']].values.ravel()
# Y.shape

In [12]:
Y.shape
X.shape

(30488, 43)

## Training and Testing Split

In [13]:
#Building a logistic regression model with default values for parameters

#apply cross validation, using 80/20 train/test splitting
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=2, test_size=0.20, random_state=0)

#apply standard scaling: to yield standard normally distributed data
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()

#apply logistic regression
from sklearn.linear_model import LogisticRegression
regEstimator = LogisticRegression()

X_train_scaled = scl_obj.fit_transform(X) # apply to training
    
    

regEstimator.fit(X_train_scaled, Y)  # train object

LogisticRegression()

In [14]:
# Separating out the features
X = X.values
X.shape
#Separating out the target
Y = Y.values.ravel()
Y.shape

(30488,)

In [15]:
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)

## Logistic Regression

In [16]:
#Building a logistic regression model with default values for parameters

#apply cross validation, using 80/20 train/test splitting
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=2, test_size=0.20, random_state=0)

#apply standard scaling: to yield standard normally distributed data
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()

from sklearn import metrics as mt
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

#apply logistic regression
from sklearn.linear_model import LogisticRegression
regEstimator = LogisticRegression()

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv.split(X,Y): 
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    
    X_test = X[test_indices]
    Y_test = Y[test_indices] 
    
    X_train_scaled = scl_obj.fit_transform(X_train) # apply to training
    X_test_scaled = scl_obj.transform(X_test) 
    
    regEstimator.fit(X_train_scaled, Y_train)  # train object
    y_hat = regEstimator.predict(X_test_scaled) # get test set precitions
    acc = mt.accuracy_score(Y_test,y_hat)
    conf = mt.confusion_matrix(Y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print('accuracy:', acc )
    print("confusion matrix\n",conf)
    #print(conf)
    iter_num+=1
    
# Also note that every time you run the above code
#   it randomly creates a new training and testing set, 
#   so accuracy will be different each time
    

# https://github.com/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


====Iteration 0  ====
accuracy: 0.9017710724827812
confusion matrix
 [[5172  178]
 [ 421  327]]
====Iteration 1  ====
accuracy: 0.9034109544112824
confusion matrix
 [[5166  160]
 [ 429  343]]


## Interpretting weights

In [17]:
# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None, solver='liblinear' ) # get object


# train the reusable logisitc regression model on the training data
lr_clf.fit(X_train,Y_train)  # train object
y_hat = lr_clf.predict(X_test) # get test set precitions

# interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = bankData2.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])
    
# does this look correct? 

age has weight of 0.002986804448735319
job has weight of 0.004523902603884739
marital has weight of -0.04925514675752567
education has weight of -0.001649833115260028
default has weight of -0.15644221289832771
housing has weight of -0.26242216004729624
loan has weight of 0.34583433465796615
contact has weight of 0.023767501225638386
month has weight of -0.21470807520634727
day_of_week has weight of -0.00639968736441983
duration has weight of -0.12658270010675196
campaign has weight of -0.019087778581731225
pdays has weight of 0.002539695529330375
previous has weight of 0.013906243310940472
poutcome has weight of 0.06406019957357074
emp.var.rate has weight of -0.0010645213505478892
cons.price.idx has weight of -0.043345300840673034
cons.conf.idx has weight of 0.03979010475719272
euribor3m has weight of 0.018951309176585236
nr.employed has weight of 0.004723955149351456


In [18]:
X_train.shape

(24390, 43)

These weight interpretations are not neccessarily interpretable because of the values we had. Very large attribute values could just as easily be assigned a higher weight. Instead, let's normalize the feature values so that all the attributes are on the same dynamic range. Once we normalize the attributes, the weights should have magnitudes that reflect their poredictive power in the logistic regression model.

In [19]:
from sklearn.preprocessing import StandardScaler

# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)

# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05, solver='liblinear') # get object, the 'C' value is less (can you guess why??)
lr_clf.fit(X_train_scaled,Y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(Y_test,y_hat)
conf = mt.confusion_matrix(Y_test,y_hat)
print('accuracy:', acc )
print(conf )

# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,bankData2.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out

accuracy: 0.9025910134470319
[[5166  160]
 [ 434  338]]
housing has weight of -1.2101307850356864
day_of_week has weight of -0.3098294751394472
education has weight of -0.19779140651253854
marital has weight of -0.09421064320282992
duration has weight of -0.0794879442653976
cons.price.idx has weight of -0.039411962238107195
default has weight of -0.03887983722575623
campaign has weight of -0.02631792149995555
age has weight of -0.02476762861571548
emp.var.rate has weight of -0.011267302261191757
nr.employed has weight of -0.001050750041491694
euribor3m has weight of 0.002811198075874412
previous has weight of 0.014187587999689484
pdays has weight of 0.018157606483939305
cons.conf.idx has weight of 0.029261916744363917
contact has weight of 0.058322977583420095
poutcome has weight of 0.0831858360380574
month has weight of 0.14900016787710227
loan has weight of 0.48261121208776697
job has weight of 1.1548816933506276


## Look at code below

In [32]:
# now let's make a pandas Series with the names and values, and plot them
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


weights = pd.Series(lr_clf.coef_[0],index=X.columns)
weights.plot(kind='bar')
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [21]:
print(len(lr_clf.coef_[0]))

43


## Support Vector Machines

In [22]:
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = Y[train_indices]
    
    X_test = X[test_indices]
    y_test = Y[test_indices]
    
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test)

# https://github.com/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb

In [23]:
# lets investigate SVMs on the data and play with the parameters and kernels
from sklearn.svm import SVC

# train the model just as before
svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto') # get object
svm_clf.fit(X_train_scaled, y_train)  # train object

y_hat = svm_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf)

accuracy: 0.8935716628402754
[[5173  107]
 [ 542  276]]


In [24]:
# look at the support vectors
print(svm_clf.support_vectors_.shape)
print(svm_clf.support_.shape)
print(svm_clf.n_support_ )

(5874, 43)
(5874,)
[3075 2799]


## Need to add weights section below here for code below to run

In [25]:
# if using linear kernel, these make sense to look at (not otherwise, why?)
print(svm_clf.coef_)
weights = pd.Series(svm_clf.coef_[0],index=bankData2.columns)
weights.plot(kind='bar')

AttributeError: coef_ is only available when using a linear kernel

In [ ]:
# if using linear kernel, these make sense to look at (not otherwise, why?)
# print(svm_clf.coef_)
# weights = pd.Series(svm_clf.coef_[0],index=df_imputed.columns)
# weights.plot(kind='bar')

In [ ]:
# Now let's do some different analysis with the SVM and look at the instances that were chosen as support vectors

# now lets look at the support for the vectors and see if we they are indicative of anything
# grabe the rows that were selected as support vectors (these are usually instances that are hard to classify)

# make a dataframe of the training data
df_tested_on = bankData2.iloc[train_indices].copy() # saved from above, the indices chosen for training
# now get the support vectors from the trained model
df_support = bankData2.iloc[svm_clf.support_,:].copy()

df_support["y"] = Y[svm_clf.support_] # add back in the 'Yes' Column to the pandas dataframe
bankData2["y"] = Y # also add it back in for the original data
df_support.info()

# https://github.com/jakemdrew/DataMiningNotebooks/blob/master/04.%20Logits%20and%20SVM.ipynb

In [ ]:
# now lets see the statistics of these attributes
from pandas.plotting import boxplot

# group the original data and the support vectors
df_grouped_support = df_support.groupby(["y"])
df_grouped = bankData2.groupby(["y"])

# plot KDE of Different variables
vars_to_plot = ['age','campaign','previous','nr.employed']

for v in vars_to_plot:
    plt.figure(figsize=(10,4))
    # plot support vector stats
    plt.subplot(1,2,1)
    ax = df_grouped_support[v].plot.kde() 
    plt.legend(["n","y"])
    plt.title(v+' (Instances chosen as Support Vectors)')
    
    # plot original distributions
    plt.subplot(1,2,2)
    ax = df_grouped[v].plot.kde() 
    plt.legend(['y','n'])
    plt.title(v+' (Original)')
    


You can also look at joint plots of the data and see how relationships have changed. 
**(Hint hint for the min-lab assignment--this would be a nice analysis of the support vectors.)**

In [ ]:
## Evaluating Model